In [2]:
from cobra.flux_analysis import pfba
from os import getcwd
import math
import numpy as np
import pandas as pd
from __future__ import unicode_literals

from operator import attrgetter
from numpy import zeros
from six import iteritems

In [3]:
from cobra.io import read_sbml_model, write_sbml_model
from cobra import Reaction, Metabolite, Model

In [4]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [5]:
relative_directory = getcwd()
mcaps = read_sbml_model(relative_directory + '/../ModelFiles/xml/iMcBath.xml')

In [6]:
mcaps.reactions.SMMOi.bounds = 0, 0
mcaps.reactions.PMMOipp.bounds = 0, 1000

## Reactions using Urea

In [7]:
mcaps.metabolites.urea_p.reactions

frozenset({<Reaction UREAabcpp at 0x117dfe240>,
           <Reaction UREAtex at 0x117df74e0>})

In [8]:
mcaps.reactions.DADK.gene_reaction_rule

'MCA2066'

In [9]:
for rxn in mcaps.metabolites.urea_c.reactions:
    print (rxn.id, rxn.gene_reaction_rule)

UREASE_GAPFILLING 
BLUB MCA2289
UREAabcpp MCA1662


In [10]:
mcaps.medium

{'EX_ca2_e': 1000.0,
 'EX_cd2_e': 1000.0,
 'EX_ch4_e': 18.46,
 'EX_cl_e': 1000.0,
 'EX_co_e': 1000.0,
 'EX_cobalt2_e': 1000.0,
 'EX_cu2_e': 1000.0,
 'EX_fe2_e': 1000.0,
 'EX_h2o_e': 1000.0,
 'EX_h_e': 1000.0,
 'EX_k_e': 1000.0,
 'EX_mg2_e': 1000.0,
 'EX_mn2_e': 1000.0,
 'EX_na1_e': 1000.0,
 'EX_ni2_e': 1000.0,
 'EX_no3_e': 1000.0,
 'EX_o2_e': 1000.0,
 'EX_pi_e': 1000.0,
 'EX_so4_e': 1000.0,
 'EX_zn2_e': 1000.0}

## Results Collection

In [11]:
df = pd.DataFrame(index=["Methane UTR","Oxygen UTR", "CO2 UTR", "NH4 UTR", "N2 UTR",
                         "NO3 UTR", "Uphill Electron Transfer Flux", "Growth Rate" ,
                         "Growth Yield", "O2/CH4 Ratio", "NH4/CH4 Ratio", "N2/CH4 Ratio",
                         "NO3/CH4 Ratio", "ATP/CH4 Ratio", "CCE"])

In [12]:
def df_record(solution, df, column):
    gr = solution.fluxes.BIOMASS_Mcapsulatus
    bc = carbon_in_biomass(mcaps.reactions.BIOMASS_Mcapsulatus)["C"]
    yx = gr/(abs((solution.fluxes.EX_ch4_e/1000)*16.04))
    data = [solution.fluxes.EX_ch4_e, solution.fluxes.EX_o2_e, solution.fluxes.EX_co2_e, solution.fluxes.EX_nh4_e,
            solution.fluxes.EX_n2_e, solution.fluxes.EX_no3_e, solution.fluxes.CYOR_q8ppi, gr, yx, 
            solution.fluxes.EX_o2_e/solution.fluxes.EX_ch4_e, solution.fluxes.EX_nh4_e/solution.fluxes.EX_ch4_e,
            solution.fluxes.EX_n2_e/solution.fluxes.EX_ch4_e, solution.fluxes.EX_no3_e/solution.fluxes.EX_ch4_e,
            solution.fluxes.ATPSipp/abs(solution.fluxes.EX_ch4_e), ((gr * bc)/ solution.fluxes.EX_ch4_e)*100]
    df[column] = data
    return df

In [13]:
mcaps.reactions.EX_n2_e.bounds = 0, 1000
mcaps.reactions.ATPSNaipp.bounds = 0, 0
mcaps.reactions.CYOR_q8ppi.bounds = 0, 1000

In [14]:
def carbon_in_biomass(biomass_rxn):
    total_biomass_formula = {}
    for met, stoich in biomass_rxn.metabolites.items():
        elements = {key: value*stoich for (key, value) in met.elements.items()}
        for key, value in elements.items():
            if key not in total_biomass_formula:
                total_biomass_formula[key] = value
            else:
                total_biomass_formula[key] += value
    return total_biomass_formula

In [15]:
def summary(model, solution):
    metabolites = {m for r in model.exchanges for m in r.metabolites}
    index = sorted(metabolites, key=attrgetter('id'))
    metabolite_fluxes = pd.DataFrame({'id': [m for m in index],'flux': zeros(len(index), dtype=float)}, index=[m.id for m in index])
    for rxn in model.exchanges:
        for met, stoich in iteritems(rxn.metabolites):
            metabolite_fluxes.at[met.id, 'flux'] += stoich * solution[rxn.id]
    print ("IN")
    print ("----------")
    for x,y in metabolite_fluxes["flux"].items():
        if round(y, 4) > 0:
            print (x,round(y, 4))
    print ("OUT")
    print ("----------")
    for x,y in metabolite_fluxes["flux"].items():
        if round(y, 4) < 0:
            print (x,round(-y, 4))
    print ("Growth Rate")
    print ("----------")
    print (solution.fluxes.BIOMASS_Mcapsulatus)

## Redox-Arm Only

Carbon-Limited, NO3, pMMO, Redox-Arm Only

In [16]:
with mcaps:
    mcaps.reactions.EX_no3_e.bounds = -1000, 0
    mcaps.reactions.GLNS.bounds = 0, 1000
    mcaps.reactions.ALAD_L.bounds = 0, 1000
    solution = pfba(mcaps)
    
    summary(mcaps, solution)

df = df_record(solution, df, 'NO3, pMMO, Redox-Arm Only')

IN
----------
ca2_e 0.0124
ch4_e 18.46
cl_e 0.0382
cu2_e 0.0003
fe2_e 0.0011
h_e 1.4581
k_e 0.0314
mg2_e 0.022
na1_e 0.007
no3_e 1.4586
o2_e 27.8071
pi_e 0.098
so4_e 0.03
zn2_e 0.0002
OUT
----------
co2_e 12.468
h2o_e 32.9602
Growth Rate
----------
0.17804211751424018


Carbon-Limited, NO3, sMMO, Redox-Arm Only

In [17]:
with mcaps:
    mcaps.reactions.EX_no3_e.bounds = -1000, 0
    mcaps.reactions.SMMOi.bounds = 0, 1000
    mcaps.reactions.PMMOipp.bounds = 0, 0
    mcaps.reactions.GLNS.bounds = 0, 1000
    mcaps.reactions.ALAD_L.bounds = 0, 1000
    solution = pfba(mcaps)
    
    summary(mcaps, solution)

df = df_record(solution, df, 'NO3, sMMO, Redox-Arm Only')

IN
----------
ca2_e 0.0124
ch4_e 18.46
cl_e 0.0382
cu2_e 0.0003
fe2_e 0.0011
h_e 1.4584
k_e 0.0314
mg2_e 0.022
na1_e 0.007
no3_e 1.4587
o2_e 27.807
pi_e 0.0981
so4_e 0.03
zn2_e 0.0002
OUT
----------
co2_e 12.4681
h2o_e 32.9606
Growth Rate
----------
0.17800416560102383


Carbon-Limited, NH3, pMMO, Redox-Arm Only

In [18]:
with mcaps:
    mcaps.reactions.EX_nh4_e.bounds = -1000, 0
    mcaps.reactions.EX_no3_e.bounds = 0, 0
    mcaps.reactions.GLNS.bounds = 0, 0
    solution = pfba(mcaps)

df = df_record(solution, df, 'NH3, pMMO, Redox-Arm Only')

Carbon-Limited, NH3, sMMO, Redox-Arm Only

In [19]:
with mcaps:
    mcaps.reactions.EX_nh4_e.bounds = -1000, 0
    mcaps.reactions.EX_no3_e.bounds = 0, 0
    mcaps.reactions.SMMOi.bounds = 0, 1000
    mcaps.reactions.PMMOipp.bounds = 0, 0
    mcaps.reactions.GLNS.bounds = 0, 0
    solution = pfba(mcaps)

df = df_record(solution, df, 'NH3, sMMO, Redox-Arm Only')

## Redox-Arm Parameter Exploration

In [20]:
mcaps.reactions.ATPSipp.metabolites

{<Metabolite adp_c at 0x117207e48>: -1.0,
 <Metabolite h_p at 0x11736d550>: -4.0,
 <Metabolite pi_c at 0x117413d68>: -1.0,
 <Metabolite atp_c at 0x117254ba8>: 1.0,
 <Metabolite h2o_c at 0x11732eb38>: 1.0,
 <Metabolite h_c at 0x11734ab70>: 3.0}

NO3

In [22]:
for r in np.arange(0,2,0.2):
    with mcaps:
        mcaps.reactions.EX_no3_e.bounds = -1000, 0
        mcaps.reactions.GLNS.bounds = 0, 1000
        mcaps.reactions.ALAD_L.bounds = 0, 1000
        mcaps.reactions.ATPSipp.add_metabolites(
            {mcaps.metabolites.adp_c: -1.0, 
             mcaps.metabolites.atp_c: 1.0,
             mcaps.metabolites.h2o_c: 1.0,
             mcaps.metabolites.h_c: (3.0-r),
             mcaps.metabolites.h_p: -(4.0-r), 
             mcaps.metabolites.pi_c: -1.0}, combine=False)
        solution = pfba(mcaps)
#         print(mcaps.reactions.ATPSipp)
#         print(solution.fluxes.ATPSipp)

    df = df_record(solution, df, 'NO3, pMMO, Redox-Arm, H/ATP {}'.format(3-r))

NH4

In [23]:
for r in np.arange(0,2,0.2):
    with mcaps:
        mcaps.reactions.EX_nh4_e.bounds = -1000, 0
        mcaps.reactions.GLNS.bounds = 0, 0
        mcaps.reactions.ALAD_L.bounds = 0, 1000
        mcaps.reactions.ATPSipp.add_metabolites(
            {mcaps.metabolites.adp_c: -1.0, 
             mcaps.metabolites.atp_c: 1.0,
             mcaps.metabolites.h2o_c: 1.0,
             mcaps.metabolites.h_c: (3.0-r),
             mcaps.metabolites.h_p: -(4.0-r), 
             mcaps.metabolites.pi_c: -1.0}, combine=False)
        solution = pfba(mcaps)
#         print(mcaps.reactions.ATPSipp)
#         print(solution.fluxes.ATPSipp)

    df = df_record(solution, df, 'NH4, pMMO, Redox-Arm, H/ATP {}'.format(3-r))

## Direct Coupling Exploration

Carbon-Limited, NO3, pMMODC

In [24]:
for r in np.arange(0,6.5,0.5):
    with mcaps:
        mcaps.reactions.EX_no3_e.bounds = -1000, 0
        mcaps.reactions.PMMOipp.bounds = 0, 1000
        mcaps.reactions.PMMODCipp.bounds = 0, 1000
        mcaps.reactions.GLNS.bounds = 0, 1000
        mcaps.reactions.ALAD_L.bounds = 0, 1000
        c3 = mcaps.problem.Constraint(
                mcaps.reactions.PMMOipp.flux_expression - 
                r* mcaps.reactions.PMMODCipp.flux_expression, lb=0, ub=0, name='c3')
        mcaps.add_cons_vars(c3)
        solution = pfba(mcaps)
        print(solution.fluxes.MEOHDHipp,solution.fluxes.PMMODCipp, solution.fluxes.PMMOipp)

    df = df_record(solution, df, 'NO3, pMMODC, RedEff {}'.format(r))

18.46 18.46 0.0
18.46 12.306666666666667 6.153333333333333
18.46 9.23 9.23
18.46 7.384000000000001 11.076000000000002
18.460000000000008 6.153333333333337 12.306666666666674
18.459999999999567 5.274285714285593 13.185714285713983
18.460000000000264 4.615000000000066 13.845000000000198
18.460000000000036 4.102222222222232 14.35777777777781
18.460000000000093 3.692000000000018 14.768000000000072
18.459999999999983 3.3563636363636338 15.103636363636351
18.459998126449538 3.076666354408257 15.383331772041283
18.459999999999887 2.8399999999999825 15.619999999999903
18.459999999999766 2.637142857142824 15.822857142856945


Carbon-Limited, NH4, pMMODC

In [25]:
for r in np.arange(0,6.5,0.5):
    with mcaps:
        mcaps.reactions.EX_nh4_e.bounds = -1000, 0
        mcaps.reactions.PMMOipp.bounds = 0, 1000
        mcaps.reactions.PMMODCipp.bounds = 0, 1000
        mcaps.reactions.GLNS.bounds = 0, 0
        mcaps.reactions.ALAD_L.bounds = 0, 1000
        c3 = mcaps.problem.Constraint(
                mcaps.reactions.PMMOipp.flux_expression - 
                r* mcaps.reactions.PMMODCipp.flux_expression, lb=0, ub=0, name='c3')
        mcaps.add_cons_vars(c3)
        solution = pfba(mcaps)
        print(solution.fluxes.MEOHDHipp,solution.fluxes.PMMODCipp, solution.fluxes.PMMOipp)

    df = df_record(solution, df, 'NH4, pMMODC, RedEff {}'.format(r))

18.46 18.46 0.0
18.46 12.306666666666667 6.153333333333333
18.46 9.23 9.23
18.46 7.384000000000001 11.076000000000002
18.46 6.153333333333333 12.306666666666667
18.46000000000013 5.274285714285752 13.185714285714381
18.459999999999994 4.614999999999998 13.844999999999995
18.459999999999944 4.102222222222212 14.357777777777741
18.459999999999923 3.6919999999999846 14.767999999999939
18.459999999999827 3.356363636363605 15.103636363636221
18.459999999999983 3.0766666666666636 15.383333333333317
18.460000000000026 2.840000000000004 15.62000000000002
18.45999999999993 2.6371428571428477 15.822857142857085


## Uphill ET Parameter Exploration

Carbon-Limited, NO3, pMMO, Range (-5 - 0) 

In [26]:
for r in np.arange(-5,0,0.2):
    with mcaps:
        mcaps.reactions.CYOR_q8ppi.bounds = r, 1000
        mcaps.reactions.EX_no3_e.bounds = -1000, 0
        mcaps.reactions.GLNS.bounds = 0, 1000
        mcaps.reactions.ALAD_L.bounds = 0, 1000
        solution = pfba(mcaps)

    df = df_record(solution, df, 'NO3, pMMO, Uphill ET, {}'.format(r))

Carbon-Limited, NH3, pMMO, Range (-5 - 0)

In [27]:
for r in np.arange(-5,0,0.2):
    with mcaps:
        mcaps.reactions.CYOR_q8ppi.bounds = r, 1000
        mcaps.reactions.EX_nh4_e.bounds = -1000, 0
        mcaps.reactions.GLNS.bounds = 0, 0
        solution = pfba(mcaps)

    df = df_record(solution, df, 'NH3, pMMO, Uphill ET, {}'.format(r))

Carbon-Limited, NO3, pMMO, Uphill Unconstrained

In [28]:
with mcaps:
    mcaps.reactions.CYOR_q8ppi.bounds = -1000, 1000
    mcaps.reactions.EX_no3_e.bounds = -1000, 0
    mcaps.reactions.GLNS.bounds = 0, 1000
    mcaps.reactions.ALAD_L.bounds = 0, 1000
    solution = pfba(mcaps)

df = df_record(solution, df, 'NO3, pMMO, Uphill Unconstrained')

Carbon-Limited, NH4, pMMO, Uphill Unconstrained

In [29]:
with mcaps:
    mcaps.reactions.CYOR_q8ppi.bounds = -1000, 1000
    mcaps.reactions.EX_nh4_e.bounds = -1000, 0
    mcaps.reactions.GLNS.bounds = 0, 0
    solution = pfba(mcaps)

df = df_record(solution, df, 'NH3, pMMO, Uphill Unconstrained')

In [31]:
df

,"NO3, pMMO, Redox-Arm Only","NO3, sMMO, Redox-Arm Only","NH3, pMMO, Redox-Arm Only","NH3, sMMO, Redox-Arm Only","NO3, pMMO, Redox-Arm, H/ATP 3.0","NO3, pMMO, Redox-Arm, H/ATP 2.8","NO3, pMMO, Redox-Arm, H/ATP 2.6","NO3, pMMO, Redox-Arm, H/ATP 2.4","NO3, pMMO, Redox-Arm, H/ATP 2.2","NO3, pMMO, Redox-Arm, H/ATP 2.0","NO3, pMMO, Redox-Arm, H/ATP 1.7999999999999998","NO3, pMMO, Redox-Arm, H/ATP 1.5999999999999999","NO3, pMMO, Redox-Arm, H/ATP 1.4","NO3, pMMO, Redox-Arm, H/ATP 1.2","NH4, pMMO, Redox-Arm, H/ATP 3.0","NH4, pMMO, Redox-Arm, H/ATP 2.8","NH4, pMMO, Redox-Arm, H/ATP 2.6","NH4, pMMO, Redox-Arm, H/ATP 2.4","NH4, pMMO, Redox-Arm, H/ATP 2.2","NH4, pMMO, Redox-Arm, H/ATP 2.0","NH4, pMMO, Redox-Arm, H/ATP 1.7999999999999998","NH4, pMMO, Redox-Arm, H/ATP 1.5999999999999999","NH4, pMMO, Redox-Arm, H/ATP 1.4","NH4, pMMO, Redox-Arm, H/ATP 1.2","NO3, pMMODC, RedEff 0.0","NO3, pMMODC, RedEff 0.5","NO3, pMMODC, RedEff 1.0","NO3, pMMODC, RedEff 1.5","NO3, pMMODC, RedEff 2.0","NO3, pMMODC, RedEff 2.5","NO3, pMMODC, RedEff 3.0","NO3, pMMODC, RedEff 3.5","NO3, pMMODC, RedEff 4.0","NO3, pMMODC, RedEff 4.5","NO3, pMMODC, RedEff 5.0","NO3, pMMODC, RedEff 5.5","NO3, pMMODC, RedEff 6.0","NH4, pMMODC, RedEff 0.0","NH4, pMMODC, RedEff 0.5","NH4, pMMODC, RedEff 1.0","NH4, pMMODC, RedEff 1.5","NH4, pMMODC, RedEff 2.0","NH4, pMMODC, RedEff 2.5","NH4, pMMODC, RedEff 3.0","NH4, pMMODC, RedEff 3.5","NH4, pMMODC, RedEff 4.0","NH4, pMMODC, RedEff 4.5","NH4, pMMODC, RedEff 5.0","NH4, pMMODC, RedEff 5.5","NH4, pMMODC, RedEff 6.0","NO3, pMMO, Uphill ET, -5.0","NO3, pMMO, Uphill ET, -4.8","NO3, pMMO, Uphill ET, -4.6","NO3, pMMO, Uphill ET, -4.3999999999999995","NO3, pMMO, Uphill ET, -4.199999999999999","NO3, pMMO, Uphill ET, -3.999999999999999","NO3, pMMO, Uphill ET, -3.799999999999999","NO3, pMMO, Uphill ET, -3.5999999999999988","NO3, pMMO, Uphill ET, -3.3999999999999986","NO3, pMMO, Uphill ET, -3.1999999999999984","NO3, pMMO, Uphill ET, -2.9999999999999982","NO3, pMMO, Uphill ET, -2.799999999999998","NO3, pMMO, Uphill ET, -2.599999999999998","NO3, pMMO, Uphill ET, -2.3999999999999977","NO3, pMMO, Uphill ET, -2.1999999999999975","NO3, pMMO, Uphill ET, -1.9999999999999973","NO3, pMMO, Uphill ET, -1.7999999999999972","NO3, pMMO, Uphill ET, -1.599999999999997","NO3, pMMO, Uphill ET, -1.3999999999999968","NO3, pMMO, Uphill ET, -1.1999999999999966","NO3, pMMO, Uphill ET, -0.9999999999999964","NO3, pMMO, Uphill ET, -0.7999999999999963","NO3, pMMO, Uphill ET, -0.5999999999999961","NO3, pMMO, Uphill ET, -0.3999999999999959","NO3, pMMO, Uphill ET, -0.19999999999999574","NH3, pMMO, Uphill ET, -5.0","NH3, pMMO, Uphill ET, -4.8","NH3, pMMO, Uphill ET, -4.6","NH3, pMMO, Uphill ET, -4.3999999999999995","NH3, pMMO, Uphill ET, -4.199999999999999","NH3, pMMO, Uphill ET, -3.999999999999999","NH3, pMMO, Uphill ET, -3.799999999999999","NH3, pMMO, Uphill ET, -3.5999999999999988","NH3, pMMO, Uphill ET, -3.3999999999999986","NH3, pMMO, Uphill ET, -3.1999999999999984","NH3, pMMO, Uphill ET, -2.9999999999999982","NH3, pMMO, Uphill ET, -2.799999999999998","NH3, pMMO, Uphill ET, -2.599999999999998","NH3, pMMO, Uphill ET, -2.3999999999999977","NH3, pMMO, Uphill ET, -2.1999999999999975","NH3, pMMO, Uphill ET, -1.9999999999999973","NH3, pMMO, Uphill ET, -1.7999999999999972","NH3, pMMO, Uphill ET, -1.599999999999997","NH3, pMMO, Uphill ET, -1.3999999999999968","NH3, pMMO, Uphill ET, -1.1999999999999966","NH3, pMMO, Uphill ET, -0.9999999999999964","NH3, pMMO, Uphill ET, -0.7999999999999963","NH3, pMMO, Uphill ET, -0.5999999999999961","NH3, pMMO, Uphill ET, -0.3999999999999959","NH3, pMMO, Uphill ET, -0.19999999999999574","NO3, pMMO, Uphill Unconstrained","NH3, pMMO, Uphill Unconstrained"
Methane UTR,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.459998,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-1.846000e+01,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-1

## 'Turning on' the uphill electron transfer

Using the value that was fit above

In [32]:
mcaps.reactions.CYOR_q8ppi.bounds = -2.8, 1000

Carbon-Limited, NO3, sMMO

In [33]:
with mcaps:
    mcaps.reactions.EX_no3_e.bounds = -1000, 0
    mcaps.reactions.SMMOi.bounds = 0, 1000
    mcaps.reactions.PMMOipp.bounds = 0, 0
    mcaps.reactions.GLNS.bounds = 0, 1000
    mcaps.reactions.ALAD_L.bounds = 0, 1000
    solution = pfba(mcaps)

df = df_record(solution, df, 'NO3, sMMO')

Carbon-Limited, NO3, pMMO

In [34]:
with mcaps:
    mcaps.reactions.EX_no3_e.bounds = -1000, 0
    mcaps.reactions.GLNS.bounds = 0, 1000
    mcaps.reactions.ALAD_L.bounds = 0, 1000
    solution = pfba(mcaps)

df = df_record(solution, df, 'NO3, pMMO')

Carbon-Limited, NH3, sMMO

In [35]:
with mcaps:
    mcaps.reactions.EX_nh4_e.bounds = -1000, 0
    mcaps.reactions.EX_no3_e.bounds = 0, 0
    mcaps.reactions.SMMOi.bounds = 0, 1000
    mcaps.reactions.PMMOipp.bounds = 0, 0
    mcaps.reactions.GLNS.bounds = 0, 0
    solution = pfba(mcaps)

df = df_record(solution, df, 'NH3, sMMO')

Carbon-Limited, NH3, pMMO

In [36]:
with mcaps:
    mcaps.reactions.EX_nh4_e.bounds = -1000, 0
    mcaps.reactions.EX_no3_e.bounds = 0, 0
    mcaps.reactions.GLNS.bounds = 0, 0
    solution = pfba(mcaps)

df = df_record(solution, df, 'NH3, pMMO')

Carbon-Limited, N2, sMMO

In [37]:
with mcaps:
    mcaps.reactions.EX_n2_e.bounds = -1000, 0
    mcaps.reactions.EX_no3_e.bounds = 0, 0    
    mcaps.reactions.SMMOi.bounds = 0, 1000
    mcaps.reactions.PMMOipp.bounds = 0, 0
    mcaps.reactions.GLNS.bounds = 0, 1000
    mcaps.reactions.ALAD_L.bounds = 0, 0
    solution = pfba(mcaps)

df = df_record(solution, df, 'N2, sMMO')

Carbon-Limited, N2, pMMO

In [38]:
with mcaps:
    mcaps.reactions.EX_n2_e.bounds = -1000, 0
    mcaps.reactions.EX_no3_e.bounds = 0, 0    
    mcaps.reactions.GLNS.bounds = 0, 1000
    mcaps.reactions.ALAD_L.bounds = 0, 0
    solution = pfba(mcaps)

df = df_record(solution, df, 'N2, pMMO')

## Direct Coupling 

Carbon-Limited, NO3, pMMODC

In [39]:
with mcaps:
    mcaps.reactions.EX_no3_e.bounds = -1000, 0
    mcaps.reactions.PMMOipp.bounds = 0, 0
    mcaps.reactions.PMMODCipp.bounds = 0, 1000
    mcaps.reactions.GLNS.bounds = 0, 1000
    mcaps.reactions.ALAD_L.bounds = 0, 1000
    mcaps.reactions.CYOR_q8ppi.bounds = 0, 1000
    solution = pfba(mcaps)

df = df_record(solution, df, 'NO3, pMMODC')

Carbon-Limited, NH3, pMMODC

In [40]:
with mcaps:
    mcaps.reactions.EX_nh4_e.bounds = -1000, 0
    mcaps.reactions.EX_no3_e.bounds = 0, 0
    mcaps.reactions.PMMOipp.bounds = 0, 0
    mcaps.reactions.PMMODCipp.bounds = 0, 1000
    mcaps.reactions.GLNS.bounds = 0, 0
    mcaps.reactions.CYOR_q8ppi.bounds = 0, 1000
    solution = pfba(mcaps)

df = df_record(solution, df, 'NH4, pMMODC')

Carbon-Limited, N2, pMMODC

In [41]:
with mcaps:
    mcaps.reactions.EX_n2_e.bounds = -1000, 0
    mcaps.reactions.PMMOipp.bounds = 0, 0
    mcaps.reactions.PMMODCipp.bounds = 0, 1000
    mcaps.reactions.GLNS.bounds = 0, 1000
    mcaps.reactions.ALAD_L.bounds = 0, 0
    mcaps.reactions.CYOR_q8ppi.bounds = 0, 1000
    solution = pfba(mcaps)

df = df_record(solution, df, 'N2, pMMODC')

## Effect of NH3 oxidation

Carbon-Limited, NH3, pMMO, NO2

In [42]:
with mcaps:
    mcaps.reactions.EX_nh4_e.bounds = -1000, 0
    mcaps.reactions.EX_no3_e.bounds = 0, 0
    mcaps.reactions.PMMOipp.bounds = 0, 1000
    c3 = mcaps.problem.Constraint(
    mcaps.reactions.EX_nh4_e.flux_expression + 2* mcaps.reactions.EX_no2_e.flux_expression, lb=0, ub=0, name='c3')
    mcaps.add_cons_vars(c3)
    mcaps.reactions.GLNS.bounds = 0, 0
    solution = pfba(mcaps)

df = df_record(solution, df, 'NH3, pMMO, 1/2*NO2')

Carbon-Limited, NH3, pMMO, NO2, Range(0-5)

In [43]:
np.arange(1,4.2,0.2)

array([1. , 1.2, 1.4, 1.6, 1.8, 2. , 2.2, 2.4, 2.6, 2.8, 3. , 3.2, 3.4,
       3.6, 3.8, 4. ])

In [44]:
for r in np.arange(1,4.2,0.2):
    with mcaps:
        mcaps.reactions.EX_nh4_e.bounds = -1000, 0
        mcaps.reactions.EX_no3_e.bounds = 0, 0
        mcaps.reactions.PMMOipp.bounds = 0, 1000
        c3 = mcaps.problem.Constraint(
        mcaps.reactions.EX_nh4_e.flux_expression + r* mcaps.reactions.EX_no2_e.flux_expression, lb=0, ub=0, name='c3')
        mcaps.add_cons_vars(c3)
        mcaps.reactions.GLNS.bounds = 0, 0
        solution = pfba(mcaps)

    df = df_record(solution, df, 'NH3, pMMO, 1/{}*NO2'.format(r))

Carbon-Limited, NO3, pMMO, NO2

In [45]:
with mcaps:
    mcaps.reactions.EX_no3_e.bounds = -1000, 0
    mcaps.reactions.EX_nh4_e.bounds = 0, 0
    mcaps.reactions.PMMOipp.bounds = 0, 1000
    c3 = mcaps.problem.Constraint(
    mcaps.reactions.EX_nh4_e.flux_expression + 2* mcaps.reactions.EX_no2_e.flux_expression, lb=0, ub=0, name='c3')
    mcaps.add_cons_vars(c3)
    mcaps.reactions.GLNS.bounds = 0, 1000
    solution = pfba(mcaps)

df = df_record(solution, df, 'NO3, pMMO, 1/2*NO2')

O2-Limited, NH3, pMMO, 0.29

In [46]:
with mcaps:
    mcaps.objective = mcaps.reactions.EX_o2_e
    mcaps.reactions.EX_nh4_e.bounds = -1000, 0
    mcaps.reactions.PMMOipp.bounds = 0, 1000
    mcaps.reactions.GLNS.bounds = 0, 0
    mcaps.reactions.BIOMASS_Mcapsulatus.bounds = 0.291, 0.291
    solution = pfba(mcaps)

df = df_record(solution, df, 'O2-Limited, NH3, pMMO, 0.29')

## 'Turning on' the fit reduced efficiency direct coupling.

In [47]:
mcaps.reactions.CYOR_q8ppi.bounds = 0, 1000

In [48]:
c3 = mcaps.problem.Constraint(
                mcaps.reactions.PMMOipp.flux_expression - 
                5.5* mcaps.reactions.PMMODCipp.flux_expression, lb=0, ub=0, name='c3')
mcaps.add_cons_vars(c3)

Carbon-Limited, NO3, pMMODC RedEf

In [49]:
with mcaps:
    mcaps.reactions.EX_no3_e.bounds = -1000, 0
    mcaps.reactions.PMMOipp.bounds = 0, 1000
    mcaps.reactions.PMMODCipp.bounds = 0, 1000
    mcaps.reactions.GLNS.bounds = 0, 1000
    mcaps.reactions.ALAD_L.bounds = 0, 1000
    mcaps.reactions.CYOR_q8ppi.bounds = 0, 1000
    solution = pfba(mcaps)

df = df_record(solution, df, 'NO3, pMMODC RedEf')

Carbon-Limited, NH3, pMMODC RedEf

In [50]:
with mcaps:
    mcaps.reactions.EX_nh4_e.bounds = -1000, 0
    mcaps.reactions.EX_no3_e.bounds = 0, 0
    mcaps.reactions.PMMOipp.bounds = 0, 1000
    mcaps.reactions.PMMODCipp.bounds = 0, 1000
    mcaps.reactions.GLNS.bounds = 0, 0
    mcaps.reactions.CYOR_q8ppi.bounds = 0, 1000
    solution = pfba(mcaps)

df = df_record(solution, df, 'NH4, pMMODC RedEf')

Carbon-Limited, NH3, pMMODC RedEf, NO2

In [51]:
with mcaps:
    mcaps.reactions.EX_nh4_e.bounds = -1000, 0
    mcaps.reactions.EX_no3_e.bounds = 0, 0
    mcaps.reactions.PMMOipp.bounds = 0, 1000
    mcaps.reactions.PMMODCipp.bounds = 0, 1000
    c4 = mcaps.problem.Constraint(
    mcaps.reactions.EX_nh4_e.flux_expression + 2* mcaps.reactions.EX_no2_e.flux_expression, lb=0, ub=0, name='c4')
    mcaps.add_cons_vars(c4)
    mcaps.reactions.GLNS.bounds = 0, 0
    solution = pfba(mcaps)

df = df_record(solution, df, 'NH3, pMMODC RedEf, 1/2*NO2')

In [52]:
with mcaps:
    mcaps.reactions.EX_no3_e.bounds = -1000, 0
    mcaps.reactions.EX_nh4_e.bounds = 0, 0
    mcaps.reactions.PMMOipp.bounds = 0, 1000
    mcaps.reactions.PMMODCipp.bounds = 0, 1000
    c4 = mcaps.problem.Constraint(
    mcaps.reactions.EX_nh4_e.flux_expression + 2* mcaps.reactions.EX_no2_e.flux_expression, lb=0, ub=0, name='c4')
    mcaps.add_cons_vars(c4)
    mcaps.reactions.GLNS.bounds = 0, 1000
    solution = pfba(mcaps)

df = df_record(solution, df, 'NO3, pMMODC RedEf, 1/2*NO2')

In [53]:
df.round(2)

,"NO3, pMMO, Redox-Arm Only","NO3, sMMO, Redox-Arm Only","NH3, pMMO, Redox-Arm Only","NH3, sMMO, Redox-Arm Only","NO3, pMMO, Redox-Arm, H/ATP 3.0","NO3, pMMO, Redox-Arm, H/ATP 2.8","NO3, pMMO, Redox-Arm, H/ATP 2.6","NO3, pMMO, Redox-Arm, H/ATP 2.4","NO3, pMMO, Redox-Arm, H/ATP 2.2","NO3, pMMO, Redox-Arm, H/ATP 2.0","NO3, pMMO, Redox-Arm, H/ATP 1.7999999999999998","NO3, pMMO, Redox-Arm, H/ATP 1.5999999999999999","NO3, pMMO, Redox-Arm, H/ATP 1.4","NO3, pMMO, Redox-Arm, H/ATP 1.2","NH4, pMMO, Redox-Arm, H/ATP 3.0","NH4, pMMO, Redox-Arm, H/ATP 2.8","NH4, pMMO, Redox-Arm, H/ATP 2.6","NH4, pMMO, Redox-Arm, H/ATP 2.4","NH4, pMMO, Redox-Arm, H/ATP 2.2","NH4, pMMO, Redox-Arm, H/ATP 2.0","NH4, pMMO, Redox-Arm, H/ATP 1.7999999999999998","NH4, pMMO, Redox-Arm, H/ATP 1.5999999999999999","NH4, pMMO, Redox-Arm, H/ATP 1.4","NH4, pMMO, Redox-Arm, H/ATP 1.2","NO3, pMMODC, RedEff 0.0","NO3, pMMODC, RedEff 0.5","NO3, pMMODC, RedEff 1.0","NO3, pMMODC, RedEff 1.5","NO3, pMMODC, RedEff 2.0","NO3, pMMODC, RedEff 2.5","NO3, pMMODC, RedEff 3.0","NO3, pMMODC, RedEff 3.5","NO3, pMMODC, RedEff 4.0","NO3, pMMODC, RedEff 4.5","NO3, pMMODC, RedEff 5.0","NO3, pMMODC, RedEff 5.5","NO3, pMMODC, RedEff 6.0","NH4, pMMODC, RedEff 0.0","NH4, pMMODC, RedEff 0.5","NH4, pMMODC, RedEff 1.0","NH4, pMMODC, RedEff 1.5","NH4, pMMODC, RedEff 2.0","NH4, pMMODC, RedEff 2.5","NH4, pMMODC, RedEff 3.0","NH4, pMMODC, RedEff 3.5","NH4, pMMODC, RedEff 4.0","NH4, pMMODC, RedEff 4.5","NH4, pMMODC, RedEff 5.0","NH4, pMMODC, RedEff 5.5","NH4, pMMODC, RedEff 6.0","NO3, pMMO, Uphill ET, -5.0","NO3, pMMO, Uphill ET, -4.8","NO3, pMMO, Uphill ET, -4.6","NO3, pMMO, Uphill ET, -4.3999999999999995","NO3, pMMO, Uphill ET, -4.199999999999999","NO3, pMMO, Uphill ET, -3.999999999999999","NO3, pMMO, Uphill ET, -3.799999999999999","NO3, pMMO, Uphill ET, -3.5999999999999988","NO3, pMMO, Uphill ET, -3.3999999999999986","NO3, pMMO, Uphill ET, -3.1999999999999984","NO3, pMMO, Uphill ET, -2.9999999999999982","NO3, pMMO, Uphill ET, -2.799999999999998","NO3, pMMO, Uphill ET, -2.599999999999998","NO3, pMMO, Uphill ET, -2.3999999999999977","NO3, pMMO, Uphill ET, -2.1999999999999975","NO3, pMMO, Uphill ET, -1.9999999999999973","NO3, pMMO, Uphill ET, -1.7999999999999972","NO3, pMMO, Uphill ET, -1.599999999999997","NO3, pMMO, Uphill ET, -1.3999999999999968","NO3, pMMO, Uphill ET, -1.1999999999999966","NO3, pMMO, Uphill ET, -0.9999999999999964","NO3, pMMO, Uphill ET, -0.7999999999999963","NO3, pMMO, Uphill ET, -0.5999999999999961","NO3, pMMO, Uphill ET, -0.3999999999999959","NO3, pMMO, Uphill ET, -0.19999999999999574","NH3, pMMO, Uphill ET, -5.0","NH3, pMMO, Uphill ET, -4.8","NH3, pMMO, Uphill ET, -4.6","NH3, pMMO, Uphill ET, -4.3999999999999995","NH3, pMMO, Uphill ET, -4.199999999999999","NH3, pMMO, Uphill ET, -3.999999999999999","NH3, pMMO, Uphill ET, -3.799999999999999","NH3, pMMO, Uphill ET, -3.5999999999999988","NH3, pMMO, Uphill ET, -3.3999999999999986","NH3, pMMO, Uphill ET, -3.1999999999999984","NH3, pMMO, Uphill ET, -2.9999999999999982","NH3, pMMO, Uphill ET, -2.799999999999998","NH3, pMMO, Uphill ET, -2.599999999999998","NH3, pMMO, Uphill ET, -2.3999999999999977","NH3, pMMO, Uphill ET, -2.1999999999999975","NH3, pMMO, Uphill ET, -1.9999999999999973","NH3, pMMO, Uphill ET, -1.7999999999999972","NH3, pMMO, Uphill ET, -1.599999999999997","NH3, pMMO, Uphill ET, -1.3999999999999968","NH3, pMMO, Uphill ET, -1.1999999999999966","NH3, pMMO, Uphill ET, -0.9999999999999964","NH3, pMMO, Uphill ET, -0.7999999999999963","NH3, pMMO, Uphill ET, -0.5999999999999961","NH3, pMMO, Uphill ET, -0.3999999999999959","NH3, pMMO, Uphill ET, -0.19999999999999574","NO3, pMMO, Uphill Unconstrained","NH3, pMMO, Uphill Unconstrained","NO3, sMMO","NO3, pMMO","NH3, sMMO","NH3, pMMO","N2, sMMO","N2, pMMO","NO3, pMMODC","NH4, pMMODC","N2, pMMODC","NH3, pMMO, 1/2*NO2","NH3, pMMO, 1/1.0*NO2","NH3, pMMO, 1/1.2*NO2","NH3, pMMO, 1/1.4*NO2","NH3, pMMO, 1/1.5999999999999999*NO2","NH3, pMMO, 1/1.7999999999999998*NO2","NH3, pMMO, 1/1.9999999999

In [54]:
df.to_csv("../ComplementaryData/SupplementTable2.csv")

In [55]:
df[['NO3, sMMO', 'NO3, pMMO', 'NO3, pMMO, 1/2*NO2', 
    'NO3, pMMODC, RedEff 5.5', 'NO3, pMMODC RedEf, 1/2*NO2', 
     'NH3, sMMO','NH3, pMMO', 'NH3, pMMO, 1/2*NO2', 
    'NH4, pMMODC RedEf', 'NH3, pMMODC RedEf, 1/2*NO2']]

,"NO3, sMMO","NO3, pMMO","NO3, pMMO, 1/2*NO2","NO3, pMMODC, RedEff 5.5","NO3, pMMODC RedEf, 1/2*NO2","NH3, sMMO","NH3, pMMO","NH3, pMMO, 1/2*NO2","NH4, pMMODC RedEf","NH3, pMMODC RedEf, 1/2*NO2"
Methane UTR,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.459998,-18.460000,-18.460000,-18.460000
Oxygen UTR,-27.025099,-26.424707,-26.424707,-26.404960,-26.404960,-27.861020,-26.486957,-29.769349,-26.467330,-29.755895
CO2 UTR,11.954014,11.559251,11.559251,11.546267,11.546267,9.699030,8.370172,9.415157,8.351189,9.398140
NH4 UTR,0.000000,0.000000,0.000000,0.000000,0.000000,-2.132753,-2.456247,-4.403717,-2.460869,-4.412003
N2 UTR,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.000000,0.000000,0.000000,0.000000
NO3 UTR,-1.583804,-1.679905,-1.679905,-1.683065,-1.683065,0.000000,0.000000,0.000000,0.000000,0.000000
Uphill Electron Transfer Flux,-1.583804,-2.800000,-2.800000,0.000000,0.000000,0.000000,-2.800000,-2.800000,0.000000,0.000000
Growth Rate,0.193276,0.205004,0.205004,0.205389,0.205389,0.260266,0.299743,0.268699,0.300307,0.269205
Growth Yield,0.652744,0.692350,0.692350,0.693653,0.693653,0.878985,1.012309,0.907466,1.014214,0.909174
O2/CH4 Ratio,1.463982,1.431458,1.431458,1.430388,1.430388,1.509264,1.434830,1.612641,1.433767,1.611912


The results for reduced-efficiency uphill electron transfer and reduced-efficiency direct-coupling are identical. Hence, either modes are possible at least in theory.

Growth on other carbon sources

In [56]:
def open_bounds():
    for met in mcaps.metabolites:
        if met.id not in ['na1_c', 'ca2_c', 'h_c', 'cl_c']:
            rxn_id = "TEST_" + met.id
            rxn = mcaps.add_boundary(met, reaction_id=rxn_id, type="custom", ub=1000, lb=0)

In [57]:
df2 = pd.DataFrame(index=["Methane UTR","Oxygen UTR", "CO2 UTR", "NH4 UTR", "N2 UTR",
                         "NO3 UTR", "Uphill Electron Transfer Flux", "Growth Rate" ,
                         "Growth Yield", "O2/CH4 Ratio", "NH4/CH4 Ratio", "N2/CH4 Ratio",
                         "NO3/CH4 Ratio", "ATP/CH4 Ratio", "CCE"])

Acetate

In [58]:
with mcaps:
    mcaps.reactions.EX_no3_e.bounds = -1000, 0
    mcaps.reactions.SMMOi.bounds = 0, 0
    mcaps.reactions.PMMOipp.bounds = 0, 0
    mcaps.reactions.GLNS.bounds = 0, 1000
    mcaps.reactions.ALAD_L.bounds = 0, 1000
    mcaps.reactions.CYOR_q8ppi.bounds = -2.8, 1000
    mcaps.add_boundary(mcaps.metabolites.ac_c, lb=-1000)
    solution = pfba(mcaps)

df2 = df_record(solution, df2, 'NO3, Acetate')

Infeasible: None (infeasible)

EtOH

In [59]:
with mcaps:
    mcaps.reactions.EX_no3_e.bounds = -1000, 0
    mcaps.reactions.SMMOi.bounds = 0, 0
    mcaps.reactions.PMMOipp.bounds = 0, 0
    mcaps.reactions.GLNS.bounds = 0, 1000
    mcaps.reactions.ALAD_L.bounds = 0, 0
    mcaps.reactions.CYOR_q8ppi.bounds = 0, 1000
    mcaps.add_boundary(mcaps.metabolites.etoh_c, lb=-1000)
    solution = pfba(mcaps)

df2 = df_record(solution, df2, 'NO3, EtOH')

/anaconda/envs/efpro2_rev/lib/python3.6/site-packages/ipykernel_launcher.py:4 RuntimeWarning: invalid value encountered in double_scalars
/anaconda/envs/efpro2_rev/lib/python3.6/site-packages/ipykernel_launcher.py:7 RuntimeWarning: divide by zero encountered in double_scalars
/anaconda/envs/efpro2_rev/lib/python3.6/site-packages/ipykernel_launcher.py:7 RuntimeWarning: invalid value encountered in double_scalars
/anaconda/envs/efpro2_rev/lib/python3.6/site-packages/ipykernel_launcher.py:8 RuntimeWarning: invalid value encountered in double_scalars
/anaconda/envs/efpro2_rev/lib/python3.6/site-packages/ipykernel_launcher.py:9 RuntimeWarning: divide by zero encountered in double_scalars
/anaconda/envs/efpro2_rev/lib/python3.6/site-packages/ipykernel_launcher.py:9 RuntimeWarning: invalid value encountered in double_scalars


Formaldehyde

In [60]:
with mcaps:
    mcaps.reactions.EX_no3_e.bounds = -1000, 1000
    mcaps.reactions.SMMOi.bounds = 0, 0
    mcaps.reactions.PMMOipp.bounds = 0, 0
    mcaps.reactions.GLNS.bounds = 0, 1000
    mcaps.reactions.ALAD_L.bounds = 0, 0
    mcaps.reactions.CYOR_q8ppi.bounds = 0, 1000
    mcaps.reactions.EX_fald_e.bounds = -1000, 1000
    solution = pfba(mcaps)

df2 = df_record(solution, df2, 'NO3, Fald')

/anaconda/envs/efpro2_rev/lib/python3.6/site-packages/ipykernel_launcher.py:4 RuntimeWarning: divide by zero encountered in double_scalars
/anaconda/envs/efpro2_rev/lib/python3.6/site-packages/ipykernel_launcher.py:7 RuntimeWarning: divide by zero encountered in double_scalars
/anaconda/envs/efpro2_rev/lib/python3.6/site-packages/ipykernel_launcher.py:7 RuntimeWarning: invalid value encountered in double_scalars
/anaconda/envs/efpro2_rev/lib/python3.6/site-packages/ipykernel_launcher.py:8 RuntimeWarning: invalid value encountered in double_scalars
/anaconda/envs/efpro2_rev/lib/python3.6/site-packages/ipykernel_launcher.py:8 RuntimeWarning: divide by zero encountered in double_scalars
/anaconda/envs/efpro2_rev/lib/python3.6/site-packages/ipykernel_launcher.py:9 RuntimeWarning: divide by zero encountered in double_scalars


CO2

In [61]:
with mcaps:
    mcaps.reactions.EX_no3_e.bounds = -1000, 1000
    mcaps.reactions.SMMOi.bounds = 0, 0
    mcaps.reactions.PMMOipp.bounds = 0, 0
    mcaps.reactions.GLNS.bounds = 0, 1000
    mcaps.reactions.ALAD_L.bounds = 0, 0
    mcaps.reactions.CYOR_q8ppi.bounds = 0, 1000
    mcaps.reactions.EX_co2_e.bounds = -1000, 1000
    solution = pfba(mcaps)
    print (solution.fluxes.BIOMASS_Mcapsulatus)

df2 = df_record(solution, df2, 'NO3, CO2')

Infeasible: None (infeasible)

Methanol

In [62]:
with mcaps:
    mcaps.reactions.EX_no3_e.bounds = -1000, 0
    mcaps.reactions.SMMOi.bounds = 0, 0
    mcaps.reactions.PMMOipp.bounds = 0, 0
    mcaps.reactions.GLNS.bounds = 0, 1000
    mcaps.reactions.ALAD_L.bounds = 0, 0
    mcaps.reactions.CYOR_q8ppi.bounds = 0, 1000
    mcaps.reactions.EX_meoh_e.bounds = -1000, 1000
    solution = pfba(mcaps)
    print (solution.fluxes.BIOMASS_Mcapsulatus)
    
df2 = df_record(solution, df2, 'NO3, Methanol')

9.6426957944305


/anaconda/envs/efpro2_rev/lib/python3.6/site-packages/ipykernel_launcher.py:4 RuntimeWarning: divide by zero encountered in double_scalars
/anaconda/envs/efpro2_rev/lib/python3.6/site-packages/ipykernel_launcher.py:7 RuntimeWarning: divide by zero encountered in double_scalars
/anaconda/envs/efpro2_rev/lib/python3.6/site-packages/ipykernel_launcher.py:7 RuntimeWarning: invalid value encountered in double_scalars
/anaconda/envs/efpro2_rev/lib/python3.6/site-packages/ipykernel_launcher.py:8 RuntimeWarning: invalid value encountered in double_scalars
/anaconda/envs/efpro2_rev/lib/python3.6/site-packages/ipykernel_launcher.py:8 RuntimeWarning: divide by zero encountered in double_scalars
/anaconda/envs/efpro2_rev/lib/python3.6/site-packages/ipykernel_launcher.py:9 RuntimeWarning: divide by zero encountered in double_scalars


In [63]:
df2

,"NO3, EtOH","NO3, Fald","NO3, Methanol"
Methane UTR,0.000000,0.000000,0.000000
Oxygen UTR,-1.864444,-118.610658,-256.336169
CO2 UTR,0.000000,420.478595,175.411599
NH4 UTR,0.000000,0.000000,0.000000
N2 UTR,0.000000,0.000000,0.000000
NO3 UTR,0.000000,-141.077544,-79.017158
Uphill Electron Transfer Flux,3.728889,214.596074,0.000000
Growth Rate,0.000000,17.216107,9.642696
Growth Yield,NaN,inf,inf
O2/CH4 Ratio,-inf,-inf,-inf


In [64]:
# NO3 - Reference

with mcaps:
    mcaps.reactions.SMMOi.bounds = 0, 1000
    mcaps.reactions.EX_no3_e.bounds = -10, 0
    mcaps.reactions.GLNS.bounds = 0, 1000
    mcaps.reactions.ALAD_L.bounds = 0, 1000
    solution = pfba(mcaps)
    summary(mcaps, solution)
    flux_map = Builder(map_json="../MetabolicMap/MethylococcusCapsulatusMetabolismMap.json", model=model, reaction_data=solution.fluxes.to_dict())
    flux_map.display_in_notebook()

IN
----------
ca2_e 0.0124
ch4_e 18.46
cl_e 0.0382
cu2_e 0.0003
fe2_e 0.0011
h_e 1.4584
k_e 0.0314
mg2_e 0.022
na1_e 0.007
no3_e 1.4587
o2_e 27.807
pi_e 0.0981
so4_e 0.03
zn2_e 0.0002
OUT
----------
co2_e 12.4681
h2o_e 32.9606
Growth Rate
----------
0.1780041656010276


NameError: name 'Builder' is not defined

In [65]:
# NH4 - Reference

with mcaps:
    mcaps.reactions.SMMOi.bounds = 0, 1000
    mcaps.reactions.EX_co2_e.bounds = 0, 1000
    mcaps.reactions.EX_nh4_e.bounds = -10, 0
    mcaps.reactions.EX_no3_e.bounds = 0, 0
    mcaps.reactions.GLNS.bounds = 0, 0
    mcaps.reactions.ALAD_L.bounds = 0, 1000
    c4 = mcaps.problem.Constraint(
    mcaps.reactions.EX_nh4_e.flux_expression + 2* mcaps.reactions.EX_no2_e.flux_expression, lb=0, ub=0, name='c4')
    mcaps.add_cons_vars(c4)
    solution = pfba(mcaps)
    summary(mcaps, solution)
    flux_map = Builder(map_json="../MetabolicMap/MethylococcusCapsulatusMetabolismMap.json", model=model, reaction_data=solution.fluxes.to_dict())
    flux_map.display_in_notebook()

IN
----------
ca2_e 0.0163
ch4_e 18.46
cl_e 0.05
cu2_e 0.0004
fe2_e 0.0014
k_e 0.0412
mg2_e 0.0288
na1_e 0.0091
nh4_e 3.8237
o2_e 30.7111
pi_e 0.1286
so4_e 0.0394
zn2_e 0.0002
OUT
----------
co2_e 10.6064
h2o_e 33.6423
h_e 9.5596
no2_e 1.9119
Growth Rate
----------
0.23331086842320223


NameError: name 'Builder' is not defined

In [66]:
# L-Alanine 

with mcaps:
    mcaps.reactions.SMMOi.bounds = 0, 1000
    mcaps.reactions.EX_no3_e.bounds = 0, 0
    mcaps.reactions.SK_ala__L_c.bounds = -10, 0
    solution = pfba(mcaps)
    summary(mcaps, solution)
#     visualize_fluxes(mcaps, solution)

IN
----------
ca2_e 0.0654
ch4_e 18.46
cl_e 0.2008
cobalt2_e 0.0001
cu2_e 0.0014
fe2_e 0.0057
h_e 2.3235
k_e 0.1653
mg2_e 0.1156
na1_e 0.0367
o2_e 34.3183
pi_e 0.5164
so4_e 0.1581
zn2_e 0.0009
OUT
----------
co2_e 16.9308
h2o_e 43.7612
nh4_e 2.3246
Growth Rate
----------
0.9366526512785871


In [67]:
# L-Aspartic acid
with mcaps:
    mcaps.reactions.SMMOi.bounds = 0, 1000
    mcaps.reactions.EX_no3_e.bounds = 0, 0
    mcaps.reactions.EX_asp__L_e.bounds = -10, 0
    solution = pfba(mcaps)
    
    summary(mcaps, solution)

IN
----------
asp__L_e 10.0
ca2_e 0.0655
ch4_e 18.46
cl_e 0.2011
cobalt2_e 0.0001
cu2_e 0.0014
fe2_e 0.0057
h_e 12.3136
k_e 0.1655
mg2_e 0.1158
na1_e 0.0367
o2_e 34.2763
pi_e 0.5171
so4_e 0.1583
zn2_e 0.0009
OUT
----------
co2_e 26.8901
h2o_e 43.7442
nh4_e 2.3147
Growth Rate
----------
0.9378599544971054


In [68]:
# L-Asparagine
with mcaps:
    mcaps.reactions.SMMOi.bounds = 0, 1000
    open_bounds()
    mcaps.reactions.EX_no3_e.bounds = 0, 0
    mcaps.reactions.SK_asn__L_c.bounds = -10, 0
    solution = pfba(mcaps)
    
    summary(mcaps, solution)

There is an `e` compartment but it does not look like it is the actual external compartment.
There is an `e` compartment but it does not look like it is the actual external compartment.


IN
----------
ca2_e 0.0676
cl_e 0.2073
cobalt2_e 0.0001
cu2_e 0.0015
fe2_e 0.0059
h_e 12.0729
k_e 0.1707
mg2_e 0.1194
na1_e 0.0379
o2_e 33.2627
pi_e 0.5331
so4_e 0.1632
zn2_e 0.001
OUT
----------
co2_c 25.9089
doxopa_c 0.001
h2o2_c 0.3106
h2o_c 33.3362
nh4_c 12.074
rdmbzi_c 0.001
urea_c 0.001
Growth Rate
----------
0.9669807719263367


In [69]:
# L-Cysteine
with mcaps:
    mcaps.reactions.EX_no3_e.bounds = 0, 0
    mcaps.reactions.SMMOi.bounds = 0, 1000
    mcaps.reactions.GLNS.bounds = 0, 1000
    mcaps.reactions.ALAD_L.bounds = 0, 1000
    mcaps.reactions.SK_cys__L_c.bounds = -10, 0
    solution = pfba(mcaps)
    summary(mcaps, solution)

IN
----------
ca2_e 0.0651
ch4_e 18.46
cl_e 0.1997
cobalt2_e 0.0001
cu2_e 0.0014
fe2_e 0.0057
k_e 0.1644
mg2_e 0.115
na1_e 0.0365
o2_e 34.7378
pi_e 0.5135
zn2_e 0.0009
OUT
----------
co2_e 17.1084
h2o_e 43.6781
h_e 7.7905
nh4_e 2.3678
Growth Rate
----------
0.931376028111823


In [70]:
# L-Glutamic acid
with mcaps:
    mcaps.reactions.EX_no3_e.bounds = 0, 0
    mcaps.reactions.SMMOi.bounds = 0, 1000
    mcaps.reactions.GLNS.bounds = 0, 1000
    mcaps.reactions.ALAD_L.bounds = 0, 1000
    mcaps.reactions.SK_glu__L_c.bounds = -10, 0
    solution = pfba(mcaps)
    
    summary(mcaps, solution)

IN
----------
ca2_e 0.0309
ch4_e 18.46
cl_e 0.0948
cu2_e 0.0007
fe2_e 0.0027
k_e 0.0781
mg2_e 0.0546
na1_e 0.0173
o2_e 28.693
pi_e 0.2439
so4_e 0.0747
zn2_e 0.0004
OUT
----------
co2_e 6.2818
h2o_e 32.3326
h_e 2.5407
nh4_e 6.3753
Growth Rate
----------
0.44233402553584455


In [71]:
# L-Glutamine
with mcaps:
    mcaps.reactions.EX_no3_e.bounds = 0, 0
    mcaps.reactions.SMMOi.bounds = 0, 1000
    mcaps.reactions.GLNS.bounds = 0, 1000
    mcaps.reactions.ALAD_L.bounds = 0, 1000
    mcaps.reactions.SK_gln__L_c.bounds = -10, 0
    solution = pfba(mcaps)
    
    summary(mcaps, solution)

IN
----------
ca2_e 0.0309
ch4_e 18.46
cl_e 0.0948
cu2_e 0.0007
fe2_e 0.0027
k_e 0.0781
mg2_e 0.0546
na1_e 0.0173
o2_e 28.693
pi_e 0.2439
so4_e 0.0747
zn2_e 0.0004
OUT
----------
co2_e 6.2818
h2o_e 22.3326
h_e 2.5407
nh4_e 16.3753
Growth Rate
----------
0.44233402553584295


In [72]:
# L-Valine
with mcaps:
    mcaps.reactions.EX_no3_e.bounds = 0, 0
    mcaps.reactions.SMMOi.bounds = 0, 1000
    mcaps.reactions.GLNS.bounds = 0, 1000
    mcaps.reactions.ALAD_L.bounds = 0, 1000
    mcaps.reactions.SK_val__L_c.bounds = -10, 0
    solution = pfba(mcaps)
    summary(mcaps, solution)

IN
----------
ca2_e 0.0303
ch4_e 18.46
cl_e 0.0928
cu2_e 0.0007
fe2_e 0.0027
k_e 0.0764
mg2_e 0.0535
na1_e 0.017
o2_e 28.6928
pi_e 0.2388
so4_e 0.0731
zn2_e 0.0004
OUT
----------
co2_e 5.5608
h2o_e 32.1789
h_e 3.2136
nh4_e 6.4511
Growth Rate
----------
0.43307768823714643
